In [1]:
using CSV,  DataFrames
using BenchmarkTools

# Imputacion de Case_Ternaria

In [176]:
periodo_anterior(x::Integer) =  x % 100 > 1  ?  x-1  : 12 + (div(x,100) -1) * 100

df = CSV.read("G:/Mi unidad/01-Maestria Ciencia de Datos/DMEyF/competencia_01_crudo.csv", DataFrame)
size(df)

(981946, 154)

In [178]:
sort!(df, [:numero_de_cliente, :foto_mes])

global periodo_ultimo = maximum( df.foto_mes )
global periodo_anteultimo = periodo_anterior( periodo_ultimo)

# assign most common class values
df.clase_ternaria = @. ifelse( df.foto_mes < periodo_anteultimo, "CONTINUA", missing )

# pre compute sequential time
periodo = @. div(df.foto_mes,100)*12 + df.foto_mes%100

global last = nrow(df)

for i in 1:last
  if df.foto_mes[i] <= periodo_anteultimo &&  i < last &&
        ( df.numero_de_cliente[i] != df.numero_de_cliente[i+1]  ||  df.numero_de_cliente[i] == df.numero_de_cliente[i+1] && periodo[i+1]  > periodo[i] +1)
          df.clase_ternaria[i] = "BAJA+1"
  end

  if df.foto_mes[i] < periodo_anteultimo &&  i+1 < last && df.numero_de_cliente[i] == df.numero_de_cliente[i+1] &&  periodo[i+1] == periodo[i] +1 &&
        ( df.numero_de_cliente[i+1] != df.numero_de_cliente[i+2]  || df.numero_de_cliente[i+1] == df.numero_de_cliente[i+2] && periodo[i+2]  > periodo[i+1] +2)
          df.clase_ternaria[i] = "BAJA+2"
  end
end



BenchmarkTools.Trial: 7 samples with 1 evaluation.
 Range (min … max):  750.308 ms … 908.056 ms  ┊ GC (min … max): 1.65% … 17.86%
 Time  (median):     765.542 ms               ┊ GC (median):    1.81%
 Time  (mean ± σ):   801.069 ms ±  73.102 ms  ┊ GC (mean ± σ):  7.01% ±  7.87%

  █                                                           ▃  
  █▁▁▁▁▇▁▁▁▇▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁█ ▁
  750 ms           Histogram: frequency by time          908 ms <

 Memory estimate: 470.33 MiB, allocs estimate: 26650399.

In [2]:
CSV.write( "G:/Mi unidad/01-Maestria Ciencia de Datos/DMEyF/competencia_01_julia01.csv", df )

UndefVarError: UndefVarError: `df` not defined

# 1. Entrenameinto

In [2]:
using CategoricalArrays


using CSV, DataFrames, Random, Statistics
using Primes
using DecisionTree, Impute
using Base.Threads

using Distributed

In [3]:
df = CSV.read("G:/Mi unidad/01-Maestria Ciencia de Datos/DMEyF/TPs/dmeyf-2024/datasets/competencia_01_julia.csv", DataFrame)

names(df)

155-element Vector{String}:
 "numero_de_cliente"
 "foto_mes"
 "active_quarter"
 "cliente_vip"
 "internet"
 "cliente_edad"
 "cliente_antiguedad"
 "mrentabilidad"
 "mrentabilidad_annual"
 "mcomisiones"
 ⋮
 "Visa_mpagado"
 "Visa_mpagospesos"
 "Visa_mpagosdolares"
 "Visa_fechaalta"
 "Visa_mconsumototal"
 "Visa_cconsumos"
 "Visa_cadelantosefectivo"
 "Visa_mpagominimo"
 "clase_ternaria"

In [4]:
count_cliente = length(unique(df.numero_de_cliente))

169481

## Registros faltante y su posible tratamiento

In [5]:
function encontrar_registros_faltantes(df)
    registros_faltantes = []
    grupos = groupby(df, :numero_de_cliente)

    for grupo in grupos
        registros_cliente = grupo.foto_mes
        registros_cliente = sort(registros_cliente)

        if length(registros_cliente) == 6 && all(diff(registros_cliente) .== 1)
            continue
        end
        if length(registros_cliente) == 5 && (registros_cliente[1] == registros_cliente[end] - 4 || registros_cliente[end] == registros_cliente[1] + 4)
            continue
        end
        if any(diff(registros_cliente) .> 1) 
            push!(registros_faltantes, (grupo.numero_de_cliente[1]))
        end
    end

    return registros_faltantes
end

registros_faltantes = encontrar_registros_faltantes(df)
println(registros_faltantes)

@benchmark encontrar_registros_faltantes(df)

Any[263751894, 304051318, 351895410, 445822700, 493399293, 555298549, 559063769, 583196648, 644687585, 675442310, 832767503, 855499233, 885810926, 893574159, 933210746, 1179226843, 1302790375, 1389447490, 1435040696, 1460648156, 1471746344, 1524390423, 1549075943]


BenchmarkTools.Trial: 38 samples with 1 evaluation.
 Range (min … max):  103.206 ms … 248.126 ms  ┊ GC (min … max): 0.00% … 56.06%
 Time  (median):     125.482 ms               ┊ GC (median):    0.00%
 Time  (mean ± σ):   131.633 ms ±  24.074 ms  ┊ GC (mean ± σ):  7.68% ± 14.27%

           █                                                     
  ▃▁▁▃▃▁▃▄▆█▆▄▁▁▁▁▁▁▁▁▁▁▁▃▃▃▁▁▁▁▁▁▃▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▃ ▁
  103 ms           Histogram: frequency by time          248 ms <

 Memory estimate: 145.84 MiB, allocs estimate: 2540128.

In [6]:
println(length(registros_faltantes))
registros_faltantes = [Int64(x) for x in registros_faltantes]

23


23-element Vector{Int64}:
  263751894
  304051318
  351895410
  445822700
  493399293
  555298549
  559063769
  583196648
  644687585
  675442310
          ⋮
  933210746
 1179226843
 1302790375
 1389447490
 1435040696
 1460648156
 1471746344
 1524390423
 1549075943

In [2]:
function EliminarVacios(df, registros_faltantes)
    print("cantidad de registros de Train: ")
    println(size(df))
    # Eliminar los registros de estos clientes del df_train
    df = df[.!in.(df.numero_de_cliente, Ref(registros_faltantes)), :]
    print("cantidad de registros de Train: ")
    println(size(df))

    return df
end

function AgregarPerdidos(df, registros_faltantes)
    print("cantidad de registros de Train: ")
    println(size(df))

    # Crear un DataFrame vacío para almacenar los registros faltantes
    df_faltantes = DataFrame()

    # Iterar sobre los registros faltantes
    for registro in registros_faltantes
        # Buscar el registro correspondiente en el DataFrame original
        registro_original = df[df.numero_de_cliente .== registro, :]

        # Agregar una copia del registro original con 'foto_mes' faltante
        registro_faltante = deepcopy(registro_original)
        registro_faltante.foto_mes .= missing

        # Agregar el registro faltante al DataFrame de registros faltantes
        df_faltantes = vcat(df_faltantes, registro_faltante)
    end

    # Agregar los registros faltantes al DataFrame original
    df = vcat(df, df_faltantes)

    print("cantidad de registros de Train: ")
    println(size(df))
    return df
end

# Dividir el DataFrame en entrenamiento y prueba
df_train = filter(row -> row.foto_mes <= 202104, df)
df_test = filter(row -> row.foto_mes == 202106, df)

#si eliminamos
df_train = EliminarVacios(df_train, registros_faltantes)

#si agregamos los registros faltnates de esos 23...
#df_train = AgregarPerdidos(df_train)


#entrenameitno y target
X_train = select(df_train, Not(:clase_ternaria))
y_train = df_train.clase_ternaria


####----------------Control Missing
# Crear un DataFrame para mostrar los resultados
resultadoX_train = DataFrame(columna = String[], valores_faltantes = Int[])


# Iterar sobre cada columna del DataFrame original
for col in names(X_train)
    # Contar los valores faltantes en la columna actual
    missing_count = sum(ismissing.(X_train[!, col]))
    
    # Añadir el resultado al DataFrame de resultados
    push!(resultadoX_train, (col, missing_count))
end

# Contar los valores faltantes en la columna actual
resultadoy_train = sum(ismissing.(y_train))

# Mostrar el resultado
println(resultadoX_train)
print("Valores faltantes en y_train: ")
println(resultadoy_train)

UndefVarError: UndefVarError: `df` not defined

# Calculo de ganancias medias.

In [5]:
### ARMADO DE ARBOL E HYPERPARAMETROS

struct ttree
    n_subfeatures::UInt
    maxdepth::UInt
    min_samples_split::UInt
    min_samples_leaf::UInt
    min_purity_increase::Float64
 end
 
 ptree = ttree(0, 7, 800, 20, 0)
 training = 0.7
 semilla = 17
 qsemillas = 10 #0

10

In [6]:
### CALCULO GANANCIA

@everywhere function  EstimarGanancia( psemilla, training, ptree )
    println("Proceso $(myid()) ejecutando EstimarGanancia con semilla $psemilla")
    ganancia_test_normalizada = -1,0

    # particion
    Random.seed!(psemilla)
    vfold = 2 .-  Int.( rand(Float64, length(dataset_clase)) .< training )

    # train_campos =  replace!( Matrix( dataset[ vfold .== 1 ,  Not(:clase_ternaria)] ), missing => 0  )

    # clase = string.(dataset[ vfold .== 1 , :clase_ternaria ])
    # datos = Matrix( dataset[ vfold .== 1 ,  Not(:clase_ternaria)] )
    # genero el modelo en training
    modelo = DecisionTree.build_tree(
        dataset_clase[ vfold .== 1 ],
        dataset_matriz[ vfold .== 1 ,:],
        ptree.n_subfeatures,
        ptree.maxdepth,
        ptree.min_samples_leaf,
        ptree.min_samples_split,
        ptree.min_purity_increase
    )

    # aplico el modelo a testing vfold = 2
    pred = apply_tree_proba(modelo, 
        dataset_matriz[ vfold .== 2 ,:],
        ["BAJA+1","BAJA+2","CONTINUA"]
    )


   ganancia_test_normalizada = sum(
        ( dataset_clase[ vfold .== 2][ (pred[:, 2 ] .> 0.025) ] .== "BAJA+2"   )
        .* 280000
        .- 7000
       ) / ( 1.0 - training )

   return  ganancia_test_normalizada
end

In [7]:
####ENTRENAMIENTO Y CALCULO DE MEDIAS

dataset = df

# restrinjo al periodo 202104
dataset = dataset[ dataset.foto_mes .== 202104, : ]

# Lamentablemente debo imputar nulos, 
#  porque la libreria DecisionTree no los soporta
dataset = Impute.substitute( dataset ) 

# formato para  DecisionTrees
dataset_clase = string.(dataset[ :, :clase_ternaria ])
dataset_matriz = Matrix( dataset[ :, Not(:clase_ternaria)] )

# elimino  dataset
dataset = Nothing

# genero la cantidad de qsemillas  nuevas semillas
Random.seed!(semilla)
semillas = rand( Primes.primes( 100000, 999999 ), qsemillas )


# vector donde almaceno los resultados
ganancia = Array{Float64}( undef, length( semillas ))


10-element Vector{Float64}:
 6.95193226747213e-310
 8.19858838473e-312
 8.198609126947e-312
 8.199063825126e-312
 8.199836726877e-312
 8.199836727114e-312
 8.19983672735e-312
 8.19992537767e-312
 8.199925377985e-312
 8.19992537862e-312

In [35]:
semillas[10]

436853

In [9]:
# calculo las  ganancias
@threads for i=1:length(semillas) 
   ganancia[i] = EstimarGanancia( semillas[i], training, ptree )
end
 
print( ganancia )
print( Statistics.mean( ganancia ) )

Proceso 1 ejecutando EstimarGanancia con semilla 852751
Proceso 1 ejecutando EstimarGanancia con semilla 688397
Proceso 1 ejecutando EstimarGanancia con semilla 704281
Proceso 1 ejecutando EstimarGanancia con semilla 409589
Proceso 1 ejecutando EstimarGanancia con semilla 938827
Proceso 1 ejecutando EstimarGanancia con semilla 562901
Proceso 1 ejecutando EstimarGanancia con semilla 969559
Proceso 1 ejecutando EstimarGanancia con semilla 395953
Proceso 1 ejecutando EstimarGanancia con semilla 479189
Proceso 1 ejecutando EstimarGanancia con semilla 436853
[1.1339999999999999e8, 1.2212666666666664e8, 1.3817999999999997e8, 1.1806666666666664e8, 1.2387666666666664e8, 1.3355999999999999e8, 1.351233333333333e8, 1.4552999999999997e8, 1.382733333333333e8, 1.2198666666666664e8]1.2901233333333333e8

## Tratameitno de Misiing en X_train

Muy burdo todo por 0 habra que evaluar mucho de esto como DM.

In [9]:
#Remplazo los valores faltantes por 0, burdo pero me deja entrenar el modelo.
X_train = coalesce.(X_train, 0)
y_train = coalesce.(y_train, "Missing")
#se quita la columna numero_de_cliente
X_train = select(X_train, Not(:numero_de_cliente))
#y_train = select(y_train, Not(:numero_de_cliente))

#Pasamos a Matriz para entrenamiento
#X_train = Matrix(X_train)
#y_train = coalesce.(y_train, "CONTINUA")
#y_train = categorical(y_train)

#Preparamos los datos para entrenar como solicita MJL
#X_train_tab = Table(X_train)
X_train_tab = DataFrame(X_train)


# Obtener los niveles de la variable objetivo
levels_y = levels(y_train)
println(levels_y)
# Convertir la variable objetivo en un vector de categorías finitas
y_train_cat_finite = CategoricalArray(y_train, levels=levels_y)


String15["BAJA+1", "BAJA+2", "CONTINUA"]


652383-element CategoricalArray{String15,1,UInt32}:
 String15("CONTINUA")
 String15("CONTINUA")
 String15("CONTINUA")
 String15("CONTINUA")
 String15("CONTINUA")
 String15("CONTINUA")
 String15("CONTINUA")
 String15("CONTINUA")
 String15("CONTINUA")
 String15("CONTINUA")
 ⋮
 String15("CONTINUA")
 String15("CONTINUA")
 String15("CONTINUA")
 String15("CONTINUA")
 String15("CONTINUA")
 String15("CONTINUA")
 String15("CONTINUA")
 String15("CONTINUA")
 String15("CONTINUA")

In [10]:
#using DecisionTree
Tree = @load DecisionTreeClassifier pkg=DecisionTree

LoadError: LoadError: UndefVarError: `@load` not defined
in expression starting at g:\Mi unidad\01-Maestria Ciencia de Datos\DMEyF\TPs\dmeyf2024\dmeyf2024\jl_notebook_cell_df34fa98e69747e1a8f8a730347b8e2f_X24sZmlsZQ==.jl:2

In [19]:
# Ajustar parámetros
max_depth = 7
min_samples_split = 250
min_samples_leaf = 100

tree = Tree(max_depth=max_depth, min_samples_split=min_samples_split, min_samples_leaf=min_samples_leaf)

# Crear una máquina con el modelo y los datos de entrenamiento
mach = machine(tree, X_train, y_train_cat_finite)

# Entrenar el modelo
@time fit!(mach)

┌ Info: Training machine(DecisionTreeClassifier(max_depth = 7, …), …).
└ @ MLJBase C:\Users\tschoppj\.julia\packages\MLJBase\7nGJF\src\machines.jl:499


 19.351890 seconds (2.29 k allocations: 799.283 MiB, 0.11% gc time)


trained Machine; caches model-specific representations of data
  model: DecisionTreeClassifier(max_depth = 7, …)
  args: 
    1:	Source @208 ⏎ Table{Union{AbstractVector{Continuous}, AbstractVector{Count}}}
    2:	Source @555 ⏎ AbstractVector{Multiclass{3}}


## Preparar df_test para aplicar el modelo

In [17]:
# Suponiendo que las columnas de características son las mismas que X_train
# y que ya has tratado los valores faltantes en df_test
X_test = select(df_test, Not(:numero_de_cliente, :clase_ternaria))

#Se eliminan missing y se remplazan por 0
X_test = coalesce.(X_test, 0)

# Realizar predicciones en el conjunto de datos de prueba
probabilidades = MLJ.predict(mach, X_test)



164876-element CategoricalDistributions.UnivariateFiniteVector{Multiclass{3}, String15, UInt32, Float64}:
 UnivariateFinite{Multiclass{3}}(BAJA+1=>0.000526, BAJA+2=>0.00123, CONTINUA=>0.998)
 UnivariateFinite{Multiclass{3}}(BAJA+1=>0.000534, BAJA+2=>0.000672, CONTINUA=>0.999)
 UnivariateFinite{Multiclass{3}}(BAJA+1=>0.0, BAJA+2=>0.0, CONTINUA=>1.0)
 UnivariateFinite{Multiclass{3}}(BAJA+1=>0.00969, BAJA+2=>0.0196, CONTINUA=>0.971)
 UnivariateFinite{Multiclass{3}}(BAJA+1=>0.0, BAJA+2=>0.014, CONTINUA=>0.986)
 UnivariateFinite{Multiclass{3}}(BAJA+1=>0.000275, BAJA+2=>0.000153, CONTINUA=>1.0)
 UnivariateFinite{Multiclass{3}}(BAJA+1=>0.0, BAJA+2=>0.00119, CONTINUA=>0.999)
 UnivariateFinite{Multiclass{3}}(BAJA+1=>0.0, BAJA+2=>0.000312, CONTINUA=>1.0)
 UnivariateFinite{Multiclass{3}}(BAJA+1=>0.000171, BAJA+2=>9.53e-5, CONTINUA=>1.0)
 UnivariateFinite{Multiclass{3}}(BAJA+1=>0.0, BAJA+2=>0.000119, CONTINUA=>1.0)
 ⋮
 UnivariateFinite{Multiclass{3}}(BAJA+1=>0.00153, BAJA+2=>0.0046, CONTINUA=>0.99

In [18]:
# Extraer las probabilidades de la clase positiva (BAJA+2)
probabilidades_positivas = [pdf(prob, "BAJA+2") for prob in probabilidades]

# Establecer el umbral
umbral = 1/40

# Realizar predicciones con el umbral
predicciones_umbral = [prob > umbral ? 1 : 0 for prob in probabilidades_positivas]

print("Predicciones de personas que dejan el Banco: ")
n_unos = sum(predicciones_umbral)
println(n_unos)

Predicciones de personas que dejan el Banco: 8514


In [12]:
# Agregar las predicciones como una nueva columna en df_test
df_test[!, :Predicted] = predicciones_umbral

# Seleccionar las columnas 'numero_de_cliente' y 'Predicted' para la exportación
resultado_exportar = select(df_test, :numero_de_cliente, :Predicted)

# Exportar a CSV
archivo_numero = "003"
CSV.write("G:/Mi unidad/01-Maestria Ciencia de Datos/DMEyF/exp/KA2001/KJulia_" * archivo_numero * ".csv", resultado_exportar)

UndefVarError: UndefVarError: `predicciones_umbral` not defined